# AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması

## İş Problemi

Facebook kısa süre önce mevcut "maximumbidding" adı verilen teklif verme türüne alternatif olarak yeni bir teklif türü olan "average bidding"’i tanıttı. Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test etmeye karar verdi veaveragebidding'in maximumbidding'den daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B testi yapmak istiyor.A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden bu A/B testinin sonuçlarını analiz etmenizi bekliyor.Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu nedenle, istatistiksel testler için Purchasemetriğine odaklanılmalıdır.

## Veri Seti Hikayesi

Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average Bidding uygulanmıştır.

**Değişkenler**

**impression:** Reklam görüntüleme sayısı
**Click:** Görüntülenen reklama tıklama sayısı
**Purchase:** Tıklanan reklamlar sonrası satın alınan ürün sayısı
**Earning:** Satın alınan ürünler sonrası elde edilen kazanç

**Proje Görevleri**

## AB Testing (Bağımsız İki Örneklem T Testi)

1. Hipotezleri Kur
2. Varsayım Kontrolü
  - 1. Normallik Varsayımı (shapiro)
  - 2. Varyans Homojenliği (levene)
3. Hipotezin Uygulanması
  - 1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi
  - 2. Varsayımlar sağlanmıyorsa mannwhitneyu testi
4. p-value değerine göre sonuçları yorumla
Not:
- Normallik sağlanmıyorsa direkt 2 numara. Varyans homojenliği sağlanmıyorsa 1 numaraya arguman girilir.
- Normallik incelemesi öncesi aykırı değer incelemesi ve düzeltmesi yapmak faydalı olabilir.

In [1]:
# Görev 1:  Veriyi Hazırlama ve Analiz Etme

import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, \
    mannwhitneyu, pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.float_format", lambda x: "%.5f" % x)

In [3]:
# Adım 1:  ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

df_control = pd.read_excel("ab_testing.xlsx", sheet_name="Control Group")
df_control.columns = ["Impression_control", "Click_control", "Purchase_control", "Earnings_control"]
df_control.head()

,Impression_control,Click_control,Purchase_control,Earnings_control
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


In [5]:
df_test = pd.read_excel("ab_testing.xlsx", sheet_name="Test Group")
df_test.columns = ["Impression_test", "Click_test", "Purchase_test", "Earnings_test"]
df_test.head()

,Impression_test,Click_test,Purchase_test,Earnings_test
0,120103.50380,3216.54796,702.16035,1939.61124
1,134775.94336,3635.08242,834.05429,2929.40582
2,107806.62079,3057.14356,422.93426,2526.24488
3,116445.27553,4650.47391,429.03353,2281.42857
4,145082.51684,5201.38772,749.86044,2781.69752


In [25]:
# Adım 2: Kontrol ve test grubu verilerini analiz ediniz.

def check_df(dataframe, df_name):
    print(f"########################### {df_name} ###########################")
    
    print("########################### DESCRIBE ###########################")
    print(dataframe.describe())
    print("########################### SHAPE ###########################")
    print(dataframe.shape)
    print("########################### NaN ###########################")
    print(dataframe.isnull().sum())
    print("########################### Info ###########################")
    print(dataframe.info())

# Kontrol
check_df(df_control, "Control Group")
    

########################### Control Group ###########################
########################### DESCRIBE ###########################
       Impression_control  Click_control  Purchase_control  Earnings_control
count            40.00000       40.00000          40.00000          40.00000
mean         101711.44907     5100.65737         550.89406        1908.56830
std           20302.15786     1329.98550         134.10820         302.91778
min           45475.94296     2189.75316         267.02894        1253.98952
25%           85726.69035     4124.30413         470.09553        1685.84720
50%           99790.70108     5001.22060         531.20631        1975.16052
75%          115212.81654     5923.80360         637.95709        2119.80278
max          147539.33633     7959.12507         801.79502        2497.29522
########################### SHAPE ###########################
(40, 4)
########################### NaN ###########################
Impression_control    0
Click_control     

In [26]:
# Test
check_df(df_test, "Test Group")

########################### Test Group ###########################
########################### DESCRIBE ###########################
       Impression_test  Click_test  Purchase_test  Earnings_test
count         40.00000    40.00000       40.00000       40.00000
mean      120512.41176  3967.54976      582.10610     2514.89073
std        18807.44871   923.09507      161.15251      282.73085
min        79033.83492  1836.62986      311.62952     1939.61124
25%       112691.97077  3376.81902      444.62683     2280.53743
50%       119291.30077  3931.35980      551.35573     2544.66611
75%       132050.57893  4660.49791      699.86236     2761.54540
max       158605.92048  6019.69508      889.91046     3171.48971
########################### SHAPE ###########################
(40, 4)
########################### NaN ###########################
Impression_test    0
Click_test         0
Purchase_test      0
Earnings_test      0
dtype: int64
########################### Info #######################

In [28]:
# Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.
df = pd.concat([df_control, df_test], axis=1)
df.head()

,Impression_control,Click_control,Purchase_control,Earnings_control,Impression_test,Click_test,Purchase_test,Earnings_test
0,82529.45927,6090.07732,665.21125,2311.27714,120103.50380,3216.54796,702.16035,1939.61124
1,98050.45193,3382.86179,315.08489,1742.80686,134775.94336,3635.08242,834.05429,2929.40582
2,82696.02355,4167.96575,458.08374,1797.82745,107806.62079,3057.14356,422.93426,2526.24488
3,109914.40040,4910.88224,487.09077,1696.22918,116445.27553,4650.47391,429.03353,2281.42857
4,108457.76263,5987.65581,441.03405,1543.72018,145082.51684,5201.38772,749.86044,2781.69752


In [29]:
# Görev 2:  A/B Testinin Hipotezinin Tanımlanması

# Adım 1: Hipotezi tanımlayınız.

# H0: M1 = M2
# H1: M2 != M2

In [30]:
# Adım 2: Kontrol ve test grubu için purchase(kazanç) ortalamalarını analiz ediniz

print(df["Purchase_control"].mean())
print("############################")
print(df["Purchase_test"].mean())


550.8940587702316
############################
582.1060966484677


In [ ]:
# GÖREV 3: Hipotez Testinin Gerçekleştirilmesi

# 1. Normallik Varsayımı

# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1:..sağlanmamaktadır.

In [36]:
# Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.Bunlar Normallik Varsayımı ve Varyans Homojenliğidir.
# Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni üzerinden ayrı ayrı test ediniz

# Normallik testi (Control)
test_stat, pvalue = shapiro(df["Purchase_control"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
if pvalue < 0.05:
    print("H0 Hipotezi REDDEDİLİR.")
else:
    print("H0 Hipotezi REDDEDİLEMEZ.")

Test Stat = 0.9773, p-value = 0.5891
H0 Hipotezi REDDEDİLEMEZ.


In [37]:
# Normallik testi (Test)
test_stat, pvalue = shapiro(df["Purchase_test"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))


if pvalue < 0.05:
    print("H0 Hipotezi REDDEDİLİR.")
else:
    print("H0 Hipotezi REDDEDİLEMEZ.")

Test Stat = 0.9589, p-value = 0.1541
H0 Hipotezi REDDEDİLEMEZ.


In [38]:
# H0: Varyanslar Homojendir
# H1: Varyanslar Homojen Değildir

test_stat, pvalue = levene(df["Purchase_control"],
                           df["Purchase_test"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

if pvalue < 0.05:
    print("H0 Hipotezi REDDEDİLİR.")
else:
    print("H0 Hipotezi REDDEDİLEMEZ.")

Test Stat = 2.6393, p-value = 0.1083
H0 Hipotezi REDDEDİLEMEZ.


In [39]:
# Normallik varsayımı sağlanıyor, Varyan homojenliği varsayımı sağlanıyor.

# H0: M1 = M2
# H1: M2 != M2

test_stat, pvalue = ttest_ind(df["Purchase_control"],
                              df["Purchase_test"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

if pvalue < 0.05:
    print("H0 Hipotezi REDDEDİLİR.")
else:
    print("H0 Hipotezi REDDEDİLEMEZ.")

Test Stat = -0.9416, p-value = 0.3493
H0 Hipotezi REDDEDİLEMEZ.
